In [17]:
import numpy as np
import math

In [18]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import colors
import matplotlib.patches as patches
import matplotlib as mpl

In [19]:
import datetime as dt
import pytz
from sgp4.earth_gravity import wgs72
from sgp4.io import twoline2rv

In [41]:
#Math functions
def Convert_Seconds_To_Hours(period_sec):
    """
    Convert from seconds to hours, minutes and seconds.
    """
    hrs = int(period_sec/(60.*60.));
    period_sec = period_sec - hrs*60.*60.;
    mins = int(period_sec/60.);
    secs = period_sec - mins*60.;
    return hrs,mins,secs 

def Convert_Datetime_to_GMST(datetime_object):
    """
    Converts a date and time in the datetime object form
    to GMST.
    """
    obj_date = datetime_object;
    julianday = Gregorian_to_JulianDate(obj_date.year,obj_date.month,obj_date.day,obj_date.hour,obj_date.minute,obj_date.second);
    theta_GMST = Calculate_GMST(julianday);
    return theta_GMST 
    
def Wrap_Angle(rotangle):
    """
    Wraps angle to fit in [0,2pi).
    """
    wrappedangle = rotangle % (2*math.pi);
    return wrappedangle
    
def ECI_to_ECEF(ECI,theta):
    ECEF = np.zeros([3],dtype=np.float64);
    # Rotating the ECI vector into the ECEF frame via the GST angle about the Z-axis
    ECEF = np.dot(C3(theta),ECI);
    return ECEF

def Gregorian_to_JulianDate(yr, mo, d, h, m, s):
    JD = 367.0*yr - int((7*(yr+ int((mo+9)/12)))/4.0) + int((275.0*mo)/9.0) + d+ 1721013.5 + ((((s/60.0)+m)/60+h)/24.0);
    return JD 
    
def Calculate_GMST(JD):
    """
    Calculates the Greenwich Mean Sidereal Time
    """
    T_UT1 = (JD - 2451545.0)/36525.0
    theta_GMST = 67310.54841 + (876600.0*60*60 + 8640184.812866)*T_UT1 + 0.093104 * T_UT1**2 - 6.2e-6 * T_UT1**3;
    
    while theta_GMST > 86400.0:
        theta_GMST = theta_GMST - 86400;

    theta_GMST = theta_GMST/240.0;
    theta_GMST = theta_GMST - 360; 
    return theta_GMST 

def C3(alpha_rad):
    C_z = np.array([[ math.cos(alpha_rad),math.sin(alpha_rad),0], 
                  [-math.sin(alpha_rad),math.cos(alpha_rad),0],
                  [                 0,                0,1]],dtype=np.float64);
    return C_z

def Carts_to_LatLon(R):
    """
    function which converts ECEF position vectors to latitude and longitude
    """
    r_delta = np.linalg.norm(R[0:1]);
    sinA = R[1]/r_delta;
    cosA = R[0]/r_delta;

    Lon = math.atan2(sinA,cosA);

    if Lon < -math.pi:
        Lon = Lon + 2*math.pi;

    Lat = math.asin(R[2]/np.linalg.norm(R));
    
    return Lat,Lon

In [42]:
 class GroundTrack:
    def __init__(self, obj_name, tle_line1, tle_line2):
        self.tle_line1 = tle_line1;
        self.tle_line2 = tle_line2;
        self.name = obj_name;
        line1 = (tle_line1);
        line2 = (tle_line2);
        '''simulation setups, variable time'''
        delta_t = 1; #seconds
        simulation_period = 95*60*2 ;#seconds, orbit period of ISS and tianhe is roughly 95 minutes, simulating two orbits
        self.timevec = np.arange(0,simulation_period+delta_t,delta_t,dtype=np.float64);
        self.lat = np.zeros([len(self.timevec)],dtype=np.float64);
        self.lon = np.zeros([len(self.timevec)],dtype=np.float64);
        self.satellite_obj = twoline2rv(line1, line2, wgs72);
    #OEs from tle
    def Print_OEs(self):
        print ('satellite number')
        print (self.satellite_obj.satnum)
        print ('t(JulianDate)')
        print (self.satellite_obj.jdsatepoch)
        print ('t(Gregorian)')
        print (self.satellite_obj.epoch)
        print ('i')
        print (math.degrees(self.satellite_obj.inclo))
        print ('Ω')
        print (math.degrees(self.satellite_obj.nodeo))
        print ('e')
        print (self.satellite_obj.ecco)
        print ('ω')
        print (math.degrees(self.satellite_obj.argpo))
        print ('M')
        print (math.degrees(self.satellite_obj.mo))
        print ("period T")
        print ((1/self.satellite_obj.no_kozai)*2*math.pi) #orbital period(minute), can get a using kepler's third law

    #private
    def Simulating_position(self):
        #setup
        sat_state = np.zeros([6,len(self.timevec)],dtype=np.float64);
        ecef = np.zeros([3,len(self.timevec)],dtype=np.float64);

        #simulation, propagation
        for index in range(1,len(self.timevec)):
            #propagating with delta time, up to two orbits
            current_time = self.timevec[index];
            hrs,mins,secs = Convert_Seconds_To_Hours(current_time + (self.satellite_obj.epoch.hour*60*60) + (self.satellite_obj.epoch.minute*60)+ self.satellite_obj.epoch.second);
            dys = self.satellite_obj.epoch.day + int(math.ceil(hrs/24)); 
            if hrs >= 24:
                hrs = hrs - 24*int(math.ceil(hrs/24)) ;

            #generating the position and velocity from time
            '''
            we dont necessarily need to know the velocity of the cubesat as
            however the Satrec.Propagate(...) returns both position and velocity 
            so it is there more as a place holder
            but knowing r,v,t gives us the opportunity to reverse-engineer the 6-OEs
            '''
            satpos,satvel = self.satellite_obj.propagate(self.satellite_obj.epoch.year,self.satellite_obj.epoch.month,dys,hrs,mins,secs+(1e-6)*self.satellite_obj.epoch.microsecond);
            sat_state[0:3,index] = np.asarray(satpos);
            sat_state[3:6,index] = np.asarray(satvel);  

            #get gmst time.
            tle_epoch_test = dt.datetime(year=self.satellite_obj.epoch.year,month=self.satellite_obj.epoch.month,day=int(dys),hour=int(hrs),minute=int(mins),second=int(secs),microsecond=0,tzinfo= pytz.utc);
            theta_GMST =  math.radians(Convert_Datetime_to_GMST(tle_epoch_test));        

            ## Rotate ECI position vector by GMST angle to get ECEF position
            theta_GMST = Wrap_Angle(theta_GMST);
            ecef[:,index] = ECI_to_ECEF(sat_state[0:3,index],theta_GMST);
            self.lat[index],self.lon[index] = Carts_to_LatLon(ecef[:,index]);
            #now we knoe the satellite's position in terms of longitude and latitude, unto plotting


In [96]:
## plot results
def Plot(data):
    obj = GroundTrack(data[0], data[1], data[2])
    obj.Simulating_position();
    obj.Print_OEs();
    print('\n')
    
    coastline_data= np.loadtxt('Coastline.txt',skiprows=1)
    w, h = plt.figaspect(0.5)
    fig = plt.figure(figsize=(w,h))
    ax = fig.gca()
    plt.rc('font', family='serif');

    fig.suptitle(obj.name + " Ground Track on "+ obj.satellite_obj.epoch.strftime('%d %B %Y'),fontsize=16)
    plt.plot(coastline_data[:,0],coastline_data[:,1],'g');
    ax.set_xlabel('Longitude',fontsize=14)
    ax.set_ylabel('Latitude',fontsize=14)
    plt.xlim(-180,180);
    plt.ylim(-90,90);
    plt.yticks([-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30,40,50,60,70,80,90]);
    plt.xticks([-180,-150,-120,-90,-60,-30,0,30,60,90,120,150,180]);

    for index in range(0,len(obj.timevec)):
        plt.plot(math.degrees(obj.lon[index]),math.degrees(obj.lat[index]),'b.',markersize=1);

    ax.grid(True);
    fig.savefig(obj.name.replace('\n', "") + ' GROUND_TRACK.pdf',format='pdf',bbox_inches='tight',pad_inches=0.01,dpi=1200);

In [102]:
space_craft_data = ["","",""]
source_file = open("Celestrak.txt", "r")
counter = 0
for lines in source_file:
    space_craft_data[counter] = lines
    if(counter == 2):
        print(space_craft_data)
        Plot(space_craft_data)
    counter = (counter+1)%3
    
    